In [11]:
import cv2
import numpy as np
import imutils

In [12]:
def get_corner(contours, corner_amount=4, max_iter = 100):
    coeff = 1
    while max_iter>0 and coeff>=0:
        max_iter -= 1

        eps = coeff * cv2.arcLength(contours, True)

        poly_approx = cv2.approxPolyDP(contours,eps, True)
        hull = cv2.convexHull(poly_approx)

        if(len(hull) == corner_amount):
            return hull
        else:
            if len(hull) > corner_amount:
                coeff += 0.01
            else:
                coeff -= 0.01
        return None

In [13]:
def recognize_board(image, contours, hierarchy):

    #find biggest contour
    big_area = 0
    big_contour = 0

    for contour in contours:
        area = cv2.contourArea(contour)
        if(area > big_area):
            big_area = area
            big_contour = contour
    #print(contours)
    #print(big_contour)

    if big_contour is None:
        return image

    corners = get_corner(big_contour, 4)

    if corners is None:
        return image
    
    rect = np.zeros((4,2), dtype='float32')

    corners = corners.reshape(4,2)

    sum = 10000
    index = 0

    for i in range(4):
        if(corners[i][0] + corners[i][1] < sum):
            sum = corners[i][0] + corners[i][1]
            index = 0
    rect[0] = corners[index]
    corners = np.delete(corners, index, 0)

    sum = 0
    for i in range(3):
        if(corners[i][0]+corners[i][1] > sum):
            sum = corners[i][0]+corners[i][1]
            index = i
    rect[2] = corners[index]
    corners = np.delete(corners, index, 0)


    if(corners[0][0] > corners[1][0]):
        rect[1] = corners[0]
        rect[3] = corners[1]  
    else:
        rect[1] = corners[1]
        rect[3] = corners[0]

    rect = rect.reshape(4,2)


    A = rect[0]
    B = rect[1]
    C = rect[2]
    D = rect[3]

    #print(A, B, C, D)


In [14]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 3)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        print('cant recieve frame')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11,2)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    board = recognize_board(thresh, contours, hierarchy)
    #all_contours = cv2.drawContours(frame.copy(), big_contour , -1, (255, 255, 0), 1)
    img = all

    # board, location = find_board(img)
    
    cv2.imshow('frame', board)
    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()